In [23]:
import pandas as pd
import numpy as np
from eda.tools import seq_to_num, prep_submit

In [11]:
df_test = pd.read_csv('data/kaggle_test.csv', index_col=0)
trie_predictions = pd.read_csv('submit/trie_predict.csv', index_col=0)
df_unsolved = df_test[~df_test.index.isin(trie_predictions.index)]
X_test = seq_to_num(df_unsolved.Sequence, target_split=False, pad=False)

In [16]:
X_test.shape

(61948,)

In [29]:
from models.pipeline import Pipeline
from models.baseline import Baseline
from models.diff_table import DiffTable
from models.linear_model import LinearModel
from models.nonlinear_model import NonLinearModel
from models.markov_chain import MarkovChain
from models.lin_reg import LinReg
from models.rnn import RNN
from preproc.filters import markov_filter, rnn_filter

In [30]:
models = [('DT', DiffTable(), None),
          ('LRR', LinearModel(), None),
          ('NLRR', NonLinearModel(), None),
          ('MC', MarkovChain(), markov_filter),
          ('LR', LinReg(), None),
          ('RNN', RNN(), rnn_filter)]

In [31]:
pipe = Pipeline(models, fallback=Baseline(), verbose=True)

In [32]:
pred = pipe.predict(X_test)

61948it [01:07, 912.16it/s]


solved by DT: 101
solved by LRR: 2399


/Users/ptyshevs/udata/int_seq/models/nonlinear_model.py:61: RuntimeWarning: overflow encountered in double_scalars
  return [x1 ** 2, x2 ** 2, x1 * x2, x1, x2]
/Users/ptyshevs/udata/int_seq/models/nonlinear_model.py:64: RuntimeWarning: overflow encountered in double_scalars
  return sum([x[0] ** 2 * solution[0], x[1] ** 2 * solution[1], x[0] * x[1] * solution[2], x[0] * solution[3],
/Users/ptyshevs/udata/int_seq/models/nonlinear_model.py:65: RuntimeWarning: overflow encountered in double_scalars
  x[1] * solution[4], solution[5]])


solved by NLRR: 94


0it [00:00, ?it/s]/Users/ptyshevs/envs/loc_env/lib/python3.6/site-packages/sklearn/linear_model/base.py:485: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)
1it [00:00,  8.52it/s]

solved by MC: 592


40401it [05:14, 123.12it/s]/Users/ptyshevs/envs/loc_env/lib/python3.6/site-packages/scipy/linalg/basic.py:1250: RuntimeWarning: overflow encountered in square
  resids = np.sum(np.abs(x[n:])**2, axis=0)
58762it [07:40, 127.68it/s]


solved by LR: 23582
solved by RNN: 4125
solved by fallback-model Baseline(function=mode): 31055


In [33]:
trie_pred = trie_predictions.Last.astype('float64')

In [34]:
full_pred = pd.concat([trie_pred, pred])[df_test.index]

In [35]:
prep_submit(full_pred)